In [76]:
import pandas as pd
import numpy as np
import datetime as dt
from sqlalchemy import create_engine, inspect

# Extract the Data

Scope of the analysis:

Identifying the average price, per night, of an Airbnb listing in New York City

Identifying the cost of a private room compared to the cost of an entire apartment, and therefore the financial gain.

Identifying the most luxurius and valuable boroughs within the City and the average difference in price between them.

In [77]:
csv_file = "./Resources/airbnb_host_info.csv"
review_df = pd.read_csv(csv_file)
review_df.head()

# Change the date type of the last_review column to datetime
#review_df["last_review"] = pd.to_datetime(review_df["last_review"])
review_df.dropna()
review_df.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [78]:
csv_file2 = "./Resources/airbnb_price.csv"
price_df = pd.read_csv(csv_file2)
price_df.head()

# new data frame with split value columns
new_price_df = price_df["nbhood_full"].str.split(", ", n = 1, expand = True)
 
# making separate first name column from new data frame
price_df["borough"]= new_price_df[0]
 
# making separate last name column from new data frame
price_df["area"]= new_price_df[1]
 
# Dropping old Name columns
price_df.drop(columns =["nbhood_full"], inplace = True)
 
# Drop na value
price_df.dropna()

# Display df
price_df.head()

,listing_id,price,borough,area
0,2595,225 dollars,Manhattan,Midtown
1,3831,89 dollars,Brooklyn,Clinton Hill
2,5099,200 dollars,Manhattan,Murray Hill
3,5178,79 dollars,Manhattan,Hell's Kitchen
4,5238,150 dollars,Manhattan,Chinatown


In [79]:
csv_file3 = "./Resources/airbnb_room_type.csv"
type_df = pd.read_csv(csv_file3)
type_df.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


# Transform the data

In [80]:
# Determine different Values in Column
type_df["room_type"].unique() 

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

In [81]:
# Cleaning and Reducing Categories
# Define Dict with the key-value pair to remap.
conv = {
        'Entire home/apt':'Entire Home/Apt','entire home/apt':'Entire Home/Apt','ENTIRE HOME/APT':'Entire Home/Apt',
        'private room' : 'Private Room', 'PRIVATE ROOM' : 'Private Room', 'Private room' : 'Private Room',
        'shared room' : 'Shared Room', 'Shared room' : 'Shared Room', 'SHARED ROOM' : 'Shared Room'
}
type_df_1=type_df.replace({"room_type": conv})

type_df["room_type"].unique()

array(['Entire home/apt', 'private room', 'Private room',
       'entire home/apt', 'PRIVATE ROOM', 'shared room',
       'ENTIRE HOME/APT', 'Shared room', 'SHARED ROOM'], dtype=object)

In [82]:
# Drop na values
type_df_1.dropna()

# Display df 
type_df_1.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire Home/Apt
1,3831,Cozy Entire Floor of Brownstone,Entire Home/Apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire Home/Apt
3,5178,Large Furnished Room Near B'way,Private Room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire Home/Apt


In [83]:
type_df_1["description"]

0                             Skylit Midtown Castle
1                   Cozy Entire Floor of Brownstone
2         Large Cozy 1 BR Apartment In Midtown East
3                   Large Furnished Room Near B'way
4                Cute & Cozy Lower East Side 1 bdrm
                            ...                    
25204    Lovely Privet Bedroom with Privet Restroom
25205                      No.2 with queen size bed
25206                               Seas The Moment
25207                 1B-1B apartment near by Metro
25208       Cozy Private Room in Bushwick, Brooklyn
Name: description, Length: 25209, dtype: object

# Loading the data

In [84]:
# Connect server and database

protocol = 'postgresql'
username = 'postgres'
password = 'password'
host = 'localhost'
port = 5432            
database_name = 'Airbnb_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [85]:
engine.table_names()

C:\Users\tharu\AppData\Local\Temp\ipykernel_28788\4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['airbnb_price', 'airbnb_room_type', 'airbnb_review']

In [86]:
type_df_1.to_sql(name='airbnb_room_type', con=engine, if_exists='append', index=False)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "pk_airbnb_room_type"
DETAIL:  Key (listing_id)=(2595) already exists.

[SQL: INSERT INTO airbnb_room_type (listing_id, description, room_type) VALUES (%(listing_id)s, %(description)s, %(room_type)s)]
[parameters: ({'listing_id': 2595, 'description': 'Skylit Midtown Castle', 'room_type': 'Entire Home/Apt'}, {'listing_id': 3831, 'description': 'Cozy Entire Floor of Brownstone', 'room_type': 'Entire Home/Apt'}, {'listing_id': 5099, 'description': 'Large Cozy 1 BR Apartment In Midtown East', 'room_type': 'Entire Home/Apt'}, {'listing_id': 5178, 'description': "Large Furnished Room Near B'way", 'room_type': 'Private Room'}, {'listing_id': 5238, 'description': 'Cute & Cozy Lower East Side 1 bdrm', 'room_type': 'Entire Home/Apt'}, {'listing_id': 5295, 'description': 'Beautiful 1br on Upper West Side', 'room_type': 'Entire Home/Apt'}, {'listing_id': 5441, 'description': 'Central Manhattan/near Bro

In [ ]:
review_df.to_sql(name='airbnb_review', con=engine, if_exists='append', index=False)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column " listing_id" of relation "airbnb_review" does not exist
LINE 1: INSERT INTO airbnb_review (" listing_id", host_name, last_re...
                                   ^

[SQL: INSERT INTO airbnb_review (" listing_id", host_name, last_review) VALUES (%(_listing_id)s, %(host_name)s, %(last_review)s)]
[parameters: ({'_listing_id': 2595, 'host_name': 'Jennifer', 'last_review': 'May 21 2019'}, {'_listing_id': 3831, 'host_name': 'LisaRoxanne', 'last_review': 'July 05 2019'}, {'_listing_id': 5099, 'host_name': 'Chris', 'last_review': 'June 22 2019'}, {'_listing_id': 5178, 'host_name': 'Shunichi', 'last_review': 'June 24 2019'}, {'_listing_id': 5238, 'host_name': 'Ben', 'last_review': 'June 09 2019'}, {'_listing_id': 5295, 'host_name': 'Lena', 'last_review': 'June 22 2019'}, {'_listing_id': 5441, 'host_name': 'Kate', 'last_review': 'June 23 2019'}, {'_listing_id': 5803, 'host_name': 'Laurie', 'last_review': 'June 24 2019'}  ... displaying

In [ ]:
price_df.to_sql(name='airbnb_price', con=engine, if_exists='append', index=False)

209